# Neural Networks Approach for Predictions of Empathy

*"Most people think empathy is something that you reserve for your life, your family or your friends but the reality is that it is an existential priority for a business. Our business is to meet unmet, unarticulated needs of our customers. There is no way that innovation is gonna come out if we don’t listen to the words and understand the true meaning behind them by being empathetic." - Satya Nadella, CEO of Microsoft.*

**Dataset Information** : The dataset used is called Young People Survey Dataset from Kaggle (*https://www.kaggle.com/miroslavsabo/young-people-survey/*). It contains 1010 rows and 150 columns.

**Problem Statement** : You are working for a non-profit that is recruiting student volunteers to help with Alzheimer's patients. You have been tasked with predicting how suitable a person is for this task by predicting how empathetic he or she is on the scale of 1 to 5. 

The various important features like preferences, interests, habits, opinions, and fears of young people. After exploring such features, a machine learning model is build and predictions are made for the young people's empathy. This project mainly consists of four parts: <br>
**1) Data Loading and Preprocessing.**<br>
**2) Feature Engineering.**<br>
**3) Model Training and Testing.**<br>
**4) Model Comparisons and Conclusion.**<br>
**4) Interesting Insights drawn from Dataset.**<br>

## Data Loading

In [ ]:
# Import statements
import copy
import time
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.svm import SVC, SVR
from IPython.display import display
from scipy.stats.stats import pearsonr
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_selection import VarianceThreshold, chi2, RFE, SelectKBest, f_classif
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict, GridSearchCV


First of all, import the datasets using pandas. The file 'importDataset' contains the code for importing.

In [ ]:
# Import the datasets (responses and columns).
responsesData = pd.read_csv('../input/responses.csv')
columnsData = pd.read_csv('../input/columns.csv')

print("Datasets loaded!")
print("Shape of the data : {0} and {1}".format(responsesData.shape, columnsData.shape))

Now, let's try to understand the data in even more detail.

In [ ]:
# Some styling..
pd.set_option('display.max_columns',150)
pd.set_option('display.max_rows',1010)
plt.style.use('bmh')

In [ ]:
print("Responses Data")
responsesData.head(n=3)

In [ ]:
print("Columns Data")
columnsData.head(n=3)

## Data Preprocessing

Now, after importing datasets, preprocessing the imported datasets is very important. The datasets include some missing values. After having look at the dataset, there are not much missing values. There could be numerous ways to handle data with missing values. <br>
1) Dropping the entire row. <br>
2) Filling the missing values with some data (mean, median, most frequent).<br><br>
Dropping the entire row from the dataset might not be the most intelligent way because the row might contain some important information needed for building our model. Also, since not many data entries are missing, we can also replace the empty cells with the most frequent entries of the columns.<br><br>
Also, while preprocessing, we need to take care of the categorical features. The dataset contains 11 categorical features. I have converted the categorical features into numeric. The approach used here is to get the unique values of the columns and assign each of the unique values with a number (1 - N) with N unique values.

In [ ]:
# Pre processing the data set provided
def preprocessingDataset(dataset):

    # Define imp from Imputer class for missing values
    imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)

    #### Preprocessing the Dataset
    music = dataset.iloc[:, 0:19]
    movies = dataset.iloc[:, 19:31]
    phobias = dataset.iloc[:, 63:73]
    interests = dataset.iloc[:, 31:63]
    health = dataset.iloc[:, 73:76]
    personal = dataset.iloc[:, 76:133]
    information = dataset.iloc[:, 140:150]
    expenditure = dataset.iloc[:, 133:140]

    """
    print(music)
    print(movies)
    print(phobias)
    print(interests)
    print(health)
    print(personal)
    print(information)
    print(spendings)
    """
    # Processing the personal
    for x in personal["Lying"]:
        if x == "never":
            personal.replace(x, 1.0, inplace=True)
        elif x == "only to avoid hurting someone":
            personal.replace(x, 2.0, inplace=True)
        elif x == "sometimes":
            personal.replace(x, 3.0, inplace=True)
        elif x == "everytime it suits me":
            personal.replace(x, 4.0, inplace=True)
        elif x == "Nan":
            personal.replace(x, np.nan, inplace=True)
        elif x == "nan":
            personal.replace(x, np.nan, inplace=True)

    for x in personal["Punctuality"]:
        if x == "i am often early":
            personal.replace(x, 3.0, inplace=True)
        elif x == "i am always on time":
            personal.replace(x, 2.0, inplace=True)
        elif x == "i am often running late":
            personal.replace(x, 1.0, inplace=True)
        elif x == "Nan":
            personal.replace(x, np.nan, inplace=True)
        elif x == "nan":
            personal.replace(x, np.nan, inplace=True)

    for x in personal["Internet usage"]:
        if x == "most of the day":
            personal.replace(x, 4.0, inplace=True)
        elif x == "few hours a day":
            personal.replace(x, 3.0, inplace=True)
        elif x == "less than an hour a day":
            personal.replace(x, 2.0, inplace=True)
        elif x == "no time at all":
            personal.replace(x, 1.0, inplace=True)
        elif x == "Nan":
            personal.replace(x, np.nan, inplace=True)
        elif x == "nan":
            personal.replace(x, np.nan, inplace=True)

    # Replace strings with numpy NaNs
    personal = personal.replace("NaN", np.nan)
    personal = personal.replace("nan", np.nan)

    # Replace missing values with most frequent values
    imp.fit(personal)
    personal_data = imp.transform(personal)

    d = personal_data[:, :]
    ind = []
    for x in range(len(personal_data)):
        ind.append(x)
    c = personal.columns.tolist()
    personal = pd.DataFrame(data=d, index=ind, columns=c)

    # Processing the health
    for x in health["Smoking"]:
        if x == "current smoker":
            health.replace(x, 1.0, inplace=True)
        elif x == "former smoker":
            health.replace(x, 2.0, inplace=True)
        elif x == "tried smoking":
            health.replace(x, 3.0, inplace=True)
        elif x == "never smoked":
            health.replace(x, 4.0, inplace=True)
        elif x == "Nan":
            health.replace(x, np.nan, inplace=True)
        elif x == "nan":
            health.replace(x, np.nan, inplace=True)

    for x in health["Alcohol"]:
        if x == "drink a lot":
            health.replace(x, 1.0, inplace=True)
        elif x == "social drinker":
            health.replace(x, 2.0, inplace=True)
        elif x == "never":
            health.replace(x, 3.0, inplace=True)
        elif x == "Nan":
            health.replace(x, np.nan, inplace=True)
        elif x == "nan":
            health.replace(x, np.nan, inplace=True)

    # Replace strings with numpy NaNs
    health = health.replace("NaN", np.nan)
    health = health.replace("nan", np.nan)

    # Replace missing values with most frequent values
    imp.fit(health)
    healthData = imp.transform(health)
    d = healthData[:, :]
    ind = []
    for x in range(len(healthData)):
        ind.append(x)
    c = health.columns.tolist()
    health = pd.DataFrame(data=d, index=ind, columns=c)

    # Processing the information
    for x in information["Gender"]:
        if x == "female":
            information.replace(x, 2.0, inplace=True)
        elif x == "male":
            information.replace(x, 1.0, inplace=True)
        elif x == "Nan":
            information.replace(x, np.nan, inplace=True)
        elif x == "nan":
            information.replace(x, np.nan, inplace=True)

    for x in information["Left - right handed"]:
        if x == "right handed":
            information.replace(x, 1.0, inplace=True)
        elif x == "left handed":
            information.replace(x, 2.0, inplace=True)
        elif x == "Nan":
            information.replace(x, np.nan, inplace=True)
        elif x == "nan":
            information.replace(x, np.nan, inplace=True)

    for x in information["Education"]:
        if x == "doctorate degree":
            information.replace(x, 6.0, inplace=True)
        elif x == "masters degree":
            information.replace(x, 5.0, inplace=True)
        elif x == "college/bachelor degree":
            information.replace(x, 4.0, inplace=True)
        elif x == "secondary school":
            information.replace(x, 3.0, inplace=True)
        elif x == "primary school":
            information.replace(x, 2.0, inplace=True)
        elif x == "currently a primary school pupil":
            information.replace(x, 1.0, inplace=True)
        elif x == "Nan":
            information.replace(x, np.nan, inplace=True)
        elif x == "nan":
            information.replace(x, np.nan, inplace=True)

    for x in information["Only child"]:
        if x == "yes":
            information.replace(x, 1.0, inplace=True)
        elif x == "no":
            information.replace(x, 2.0, inplace=True)
        elif x == "Nan":
            information.replace(x, np.nan, inplace=True)
        elif x == "nan":
            information.replace(x, np.nan, inplace=True)

    for x in information["Village - town"]:
        if x == "village":
            information["Village - town"].replace(x, 1.0, inplace=True)
        elif x == "city":
            information["Village - town"].replace(x, 2.0, inplace=True)
        elif x == "Nan":
            information.replace(x, np.nan, inplace=True)
        elif x == "nan":
            information.replace(x, np.nan, inplace=True)

    for x in information["House - block of flats"]:
        if x == "block of flats":
            information["House - block of flats"].replace(x, 1, inplace=True)
        elif x == "house/bungalow":
            information["House - block of flats"].replace(x, 2, inplace=True)
        elif x == "Nan":
            information.replace(x, np.nan, inplace=True)
        elif x == "nan":
            information.replace(x, np.nan, inplace=True)

    information = information.replace("nan", np.nan)
    information = information.replace("NaN", np.nan)

    imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
    imp.fit(information)
    informationData = imp.transform(information)
    d = informationData[:, :]
    ind = []
    for x in range(len(informationData)):
        ind.append(x)
    c = information.columns.tolist()
    information = pd.DataFrame(data=d, index=ind, columns=c)

    # Processing music
    music = music.replace("nan", np.nan)
    music = music.replace("NaN", np.nan)

    imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
    imp.fit(music)
    musicData = imp.transform(music)
    d = musicData[:, :]
    ind = []
    for x in range(len(musicData)):
        ind.append(x)
    c = music.columns.tolist()
    music = pd.DataFrame(data=d, index=ind, columns=c)

    # Processing movies
    movies = movies.replace("nan", np.nan)
    movies = movies.replace("NaN", np.nan)

    imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
    imp.fit(movies)
    moviesData = imp.transform(movies)
    d = moviesData[:, :]
    ind = []
    for x in range(len(moviesData)):
        ind.append(x)
    c = movies.columns.tolist()
    movies = pd.DataFrame(data=d, index=ind, columns=c)

    # Processing phobias
    phobias = phobias.replace("nan", np.nan)
    phobias = phobias.replace("NaN", np.nan)

    imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
    imp.fit(phobias)
    phobiasData = imp.transform(phobias)
    d = phobiasData[:, :]
    ind = []
    for x in range(len(phobiasData)):
        ind.append(x)
    c = phobias.columns.tolist()
    phobias = pd.DataFrame(data=d, index=ind, columns=c)

    # Processing interests
    interests = interests.replace("nan", np.nan)
    interests = interests.replace("NaN", np.nan)

    imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
    imp.fit(interests)
    interestsData = imp.transform(interests)
    d = interestsData[:, :]
    ind = []
    for x in range(len(interestsData)):
        ind.append(x)
    c = interests.columns.tolist()
    interests = pd.DataFrame(data=d, index=ind, columns=c)

    # Processing spendings
    expenditure = expenditure.replace("nan", np.nan)
    expenditure = expenditure.replace("NaN", np.nan)

    imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
    imp.fit(expenditure)
    expenditureData = imp.transform(expenditure)
    d = expenditureData[:, :]
    ind = []
    for x in range(len(expenditureData)):
        ind.append(x)
    c = expenditure.columns.tolist()
    expenditure = pd.DataFrame(data=d, index=ind, columns=c)

    # Joining all the processed sections
    joinedDatasets = music.join(movies.join(phobias.join(interests.join(health.join(personal.join(information.join(expenditure)))))))

    return joinedDatasets

In [ ]:
# Collect the dataset with missing values filled with the most frequent entry in the column. 

print("Preprocessing data might take some time..")
print("1) Missing values are being handled!")
print("2) Categorical entries are getting converted to numeric values!")
print("3) Dummy variables are being handled!")
filledData = preprocessingDataset(responsesData)
print("Done preprocessing data!")

Below is the preprocessed dataset! Have a look at it and see that the categorical feature entries are converted to numeric values.

In [ ]:
filledData.head(n=3)

Note that some models require the data to be normalized. So, let's also scale the preprocessed dataset ahead.

In [ ]:
# Scale the dataset.
def scalingDataset(dataset):
    # Scaling the dataset
    scaler = StandardScaler()
    scaledDataarray = scaler.fit_transform(dataset)
    if type(dataset) is np.ndarray:
        return scaledDataarray
    else:
        d = scaledDataarray[:, :]
        ind = []
        for x in range(len(dataset)):
            ind.append(x)
        c = dataset.columns.tolist()
        scaledData = pd.DataFrame(data=d, index=ind, columns=c)
        return scaledData

scaledData = scalingDataset(filledData)
scaledData.head(n=3)

## Data Visualization

Another important thing to note about such huge datasets is that it is interesting to explore the multicollinearity between features and try to draw some interesting conclusions from the observations.

In [ ]:
relations = filledData.corr()
# The line of code below is used from a question posted in StackOverflow
# Link : https://stackoverflow.com/questions/17778394/list-highest-correlation-pairs-from-a-large-correlation-matrix-in-pandas
multicollinearity = (relations.where(np.triu(np.ones(relations.shape), k=1).astype(np.bool)).stack().sort_values(ascending=False))

In [ ]:
# Top positive correlations are:
multicollinearity.head(n=15)

In [ ]:
# Top negative correlations are:
multicollinearity.tail(n=15)

As you can see, there are many features related to gender. Most correlated features are height, weight, PC, Cars, Action and War. Most of them are very clear to understant like men are more into war, action and cars. The person who likes Biology would also most likely, like Maths and Chemistry.

But there are some features which left me puzzled like how can 'gender be related to western movies' and 'life struggles be related to height and weight'. I did some research about it which I would like to disclose it at the end. To know the mystery behind what's going on, keep reading.......

Let's continue our visualization process. Following information would help us identify the columns which are very biased could add unnecesary bias in our model.

In [ ]:
filledData.describe()

## Feature Engineering

The most important step in our project is to select the most important features out of 150 features in order to reduce the computation time and complexity. This also helps us in know what features are totally irrelevant to our target variable "Empathy". I have used correlations between several features and the most correlated features are given as the most important features.

In [ ]:
def correlationFigure(featureVariablesMain, targetVariable):
    # Calculate correlation
    #print(featureVariablesMain.columns)
    #print(targetVariable.values)
    def correlationCalculation(targetVariable, featureVariables, features):
        columns = [] # For maintaining the feature names
        values = [] # For maintaining the corr values of features with "Empathy"

        # Traverse through all the input features
        for x in features:
            if x is not None:
                columns.append(x) # Append the column name
                # Calculate the correlation
                c = np.corrcoef(featureVariables[x], featureVariables[targetVariable])
                absC = abs(c) # Absolute value because important values might miss
                values.append(absC[0,1])

        corrValues = pd.DataFrame()
        dataDict = {'features': columns, 'correlation_values': values}
        corrValues = pd.DataFrame(dataDict)
        # Sort the value by correlation values
        sortedCorrValues = corrValues.sort_values(by="correlation_values")

        # Plot the graph to show the features with their correlation values
        figure, ax = plt.subplots(figsize=(15, 45), squeeze=True)
        ax.set_title("Correlation Coefficients of Features")
        sns.barplot(x=sortedCorrValues.correlation_values, y=sortedCorrValues['features'], ax=ax)
        ax.set_ylabel("-----------Corr Coefficients--------->")


        plt.show()

        return sortedCorrValues

    # Make a list of columns
    columns = []
    for x in featureVariablesMain.columns:
        columns.append(x)
    # Remove "Empathy" from df
    columns.remove(targetVariable)

    # Compute correlations
    correlations = correlationCalculation(targetVariable, featureVariablesMain, columns)
    return correlations

In [ ]:
# Plotting the correlations with respect to "Empathy" variable
target = "Empathy"
targetVariable = filledData['Empathy'].to_frame()
corrData = correlationFigure(scaledData, target)
importantFeatures = corrData.sort_values(by="correlation_values", ascending=True).tail(20)

In [ ]:
# 20 most important feature with their correlation values
importantFeatures

Now, let's join all the important features to make a completely prepared dataset.

In [ ]:
finalColumnsList = []
for x in importantFeatures['features']:
    finalColumnsList.append(x)

df = pd.DataFrame() # Final prepared dataset for modelling
df = filledData[finalColumnsList[0]].to_frame()
for x in range(1, len(finalColumnsList)):
    df = df.join(filledData[finalColumnsList[x]].to_frame())

## Model Building 

The final prepared data should be split into training and testing sets. We train our classifiers on the training set.

In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(df, targetVariable, test_size=0.2, random_state=0)
xTrain = xTrain.sort_index()
xTest = xTest.sort_index()
yTrain = yTrain.sort_index()
yTest = yTest.sort_index()

### Decision Tree Model

Let's try to predict the empathy using Decision Trees. The accuracy comes around 40%.

In [ ]:
# Decision Tree Modelling
def dt(xTrain, yTrain, xTest, yTest):
    print("Hyperparameter Tuning!")
    gridClassifier = DecisionTreeClassifier()
    depthList = [1,3,5,10]
    parameters = {'max_depth':depthList}
    gridSearch = GridSearchCV(estimator=gridClassifier,
                              param_grid=parameters,
                              scoring="accuracy",
                              cv=10,
                              n_jobs=5)
    gridSearch.fit(xTrain, yTrain.values.ravel())
    bestAccuracyMLP = gridSearch.best_score_
    bestParametersMLP = gridSearch.best_params_

    print("The best parameters for Decision Tree model are :\n{}\n".format(bestParametersMLP))
    dtclassifier = DecisionTreeClassifier(max_depth=3)
    dtclassifier.fit(xTrain, yTrain.values.ravel())
    yPredictiondtTest = dtclassifier.predict(xTest)
    yPredictiondtTrain = dtclassifier.predict(xTrain)
    print("Decision Tree Evaluations :\n")
    print("Training Accuracy => {}".format(accuracy_score(yTrain, yPredictiondtTrain) * 100))
    print("Testing Accuracy => {}\n".format(accuracy_score(yTest, yPredictiondtTest) * 100))
    print("Confusion Matrix => \n{}\n".format(confusion_matrix(yTest, yPredictiondtTest)))
    print("Classification Summary => \n{}\n".format(classification_report(yTest, yPredictiondtTest)))
    plt.scatter(yTest, yPredictiondtTest)

dt(xTrain, yTrain, xTest, yTest)

Let's try to plot the learning curve for the Decision Tree model.

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve

# Below function is taken from the official documentation of "sklearn" 
# Link : http://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


title = "Learning Curves"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

estimator = DecisionTreeClassifier(max_depth=3)
plot_learning_curve(estimator, title, xTrain, yTrain, cv=cv, n_jobs=4)

### KNN Model

Let's try to predict using K-Nearest Neighbours model. Clearly, KNN fails miserably and the accuracy is 30.7%.

In [ ]:
# KNN Modelling
def knn(xTrain, yTrain, xTest, yTest):
    print("Hyperparameter Tuning!")
    gridClassifier = KNeighborsClassifier()
    nearestNeighbors = [1, 3, 5, 10]
    parameters = {'n_neighbors': nearestNeighbors}
    gridSearch = GridSearchCV(estimator=gridClassifier,
                              param_grid=parameters,
                              scoring="accuracy",
                              cv=10,
                              n_jobs=5)
    gridSearch.fit(xTrain, yTrain.values.ravel())
    bestAccuracyMLP = gridSearch.best_score_
    bestParametersMLP = gridSearch.best_params_

    print("The best parameters for KNN model are :\n{}\n".format(bestParametersMLP))
    knnClassifier = KNeighborsClassifier(n_neighbors=1)
    knnClassifier.fit(xTrain, yTrain.values.ravel())
    yPredKNNTest = knnClassifier.predict(xTest)
    yPredKNNTrain = knnClassifier.predict(xTrain)
    print("KNN Evaluation :\n")
    print("Training Accuracy => {}".format(accuracy_score(yTrain, yPredKNNTrain) * 100))
    print("Testing Accuracy => {}\n".format(accuracy_score(yTest, yPredKNNTest) * 100))
    print("Confusion Matrix => \n{}\n".format(confusion_matrix(yTest, yPredKNNTest)))
    print("Classification Summary => \n{}\n".format(classification_report(yTest, yPredKNNTest)))
    plt.scatter(yTest, yPredKNNTest)

knn(xTrain, yTrain, xTest, yTest)

Let's try to plot the learning curves for KNN model.

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve

# Below function is taken from the official documentation of "sklearn" 
# Link : http://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


title = "Learning Curves"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

estimator = KNeighborsClassifier(n_neighbors=1)
plot_learning_curve(estimator, title, xTrain, yTrain, cv=cv, n_jobs=4)

### Logistic Regression

Let's try to predict using Logistic Regression model. 

In [ ]:
# Logistics Regression Modelling
def logisticRegression(xTrain, yTrain, xTest, yTest):
    logRegClassifier = LogisticRegression(multi_class='ovr', random_state=0, C=3)
    logRegClassifier.fit(xTrain, yTrain.values.ravel())
    yPredLogTest = logRegClassifier.predict(xTest)
    yPredLogTrain = logRegClassifier.predict(xTrain)
    print("Logistic Regression Evaluation :\n")
    print("Testing Accuracy => {}".format(accuracy_score(yTest, yPredLogTest) * 100))
    print("Confusion Matrix => \n{}\n".format(confusion_matrix(yTest, yPredLogTest)))
    print("Classification Summary => \n{}\n".format(classification_report(yTest, yPredLogTest)))
    plt.scatter(yTest, yPredLogTest)


logisticRegression(xTrain, yTrain, xTest, yTest)

Let's look at the learning curve of Logistic Regression.

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve

# Below function is taken from the official documentation of "sklearn" 
# Link : http://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


title = "Learning Curves"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

estimator = LogisticRegression(multi_class='ovr', random_state=0, C=3)
plot_learning_curve(estimator, title, xTrain, yTrain, cv=cv, n_jobs=4)

### Logistic Regression (SelectKBest Feature Method)

Let's try to use Logistic Regression by using SelectKBest method from sklearn library.

In [ ]:
def kBestLogReg(filledData):
    bestDF = filledData.drop(columns=['Empathy'], axis=1)
    targetVariable = filledData['Empathy'].to_frame()
    selected = SelectKBest(score_func=f_classif, k=20)
    selectedFit = selected.fit(bestDF, targetVariable.values.ravel())
    selectedFitTransform = selectedFit.transform(bestDF)

    xTrainBestK, xTestBestK, yTrainBestK, yTestBestK = train_test_split(
        scalingDataset(selectedFitTransform),
        targetVariable,
        test_size=0.2,
        random_state=0)
    print("Cross Validating for best parameters..")
    print("This might take some time..\n")
    lr = LogisticRegression(multi_class='ovr')
    cList = [10, 100, 1000, 10000]
    solverList = ['lbfgs', 'sag', 'saga', 'newton-cg']
    maxIterList = [100, 1000, 10000]
    parameters = {'C': cList, 'solver': solverList, 'max_iter': maxIterList}
    gridSearch = GridSearchCV(estimator=lr,
                              param_grid=parameters,
                              scoring="accuracy",
                              cv=10,
                              n_jobs=4)
    gridSearch.fit(xTrainBestK, yTrainBestK.values.ravel())
    bestAccuracyLogBestK = gridSearch.best_score_
    bestParametersLogBestK = gridSearch.best_params_
    print("The best parameters for Logistic Regression model are :\n{}\n".format(bestParametersLogBestK))
    # Best parameters : C:10, maxiter:100, solver:sag
    lr = LogisticRegression(C=10, max_iter=100, solver='lbfgs', multi_class='ovr', random_state=1)
    lr.fit(xTrainBestK, yTrainBestK.values.ravel())
    yPredLogBestTest = lr.predict(xTestBestK)
    bestKLogAcc = accuracy_score(yTestBestK, yPredLogBestTest)
    print("Logistic Regression using SelectKBest Method Evaluations :\n")
    print("Training Accuracy : {}".format(bestAccuracyLogBestK*100))
    print("Testing Accuracy  : {}\n".format(bestKLogAcc*100))
    print("Confusion Matrix => \n{}\n".format(confusion_matrix(yTestBestK, yPredLogBestTest)))
    print("Classification Summary => \n{}\n".format(classification_report(yTestBestK, yPredLogBestTest)))
    plt.scatter(yTestBestK, yPredLogBestTest)

kBestLogReg(filledData)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve

# Below function is taken from the official documentation of "sklearn" 
# Link : http://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


title = "Learning Curves"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

estimator = LogisticRegression(C=10, max_iter=100, solver='lbfgs', multi_class='ovr', random_state=1)
plot_learning_curve(estimator, title, xTrain, yTrain, cv=cv, n_jobs=4)

### Multilayer Perceptron

 Let's predict the empathy using Multilayer Perceptron. <br>
 *Note: While using Multilayer Perceptron, it is needed to have normalized dataset.*

In [ ]:
def mlp(df, scaledData, targetVariable):

    # For MLP, we have to use scaled data
    scaledDF = scalingDataset(df)
    xTrain, xTest, yTrain, yTest = train_test_split(scaledData, targetVariable, test_size=0.2, random_state=0, shuffle=False)
    xTrain = xTrain.sort_index()
    xTest = xTest.sort_index()
    yTrain = yTrain.sort_index()
    yTest = yTest.sort_index()
    
    # You can run the below commented code to cross validate for the best parameters
    # print("SIT BACK AND RELAX! CROSS VALIDATION WOULD TAKE SOME TIME...")
    # mlpClass = MLPClassifier(random_state=0)
    # iterList = [100, 500]
    # hiddenLayerList = [(100, 100), (100, 200)]
    # parameters = {'alpha': 10.0 ** -np.arange(1, 7), 'max_iter': iterList, 'hidden_layer_sizes': hiddenLayerList}
    # gridSearch = GridSearchCV(estimator=mlpClass,
    #                           param_grid=parameters,
    #                           scoring="accuracy",
    #                           cv=10,
    #                           n_jobs=10)
    # gridSearch.fit(xTrain, yTrain.values.ravel())
    # bestAccuracyMLP = gridSearch.best_score_
    # bestParametersMLP = gridSearch.best_params_
    # print("The best parameters for MLP model are :\n{}\n".format(bestParametersMLP))

    mlpClass = MLPClassifier(hidden_layer_sizes=(100, 200), alpha=0.1, max_iter=500, random_state=0)
    mlpClass.fit(xTrain, yTrain.values.ravel())
    yPredMLP = mlpClass.predict(xTest)
    print("Multilayer Perceptron Evaluation :\n")
    # print("Training Accuracy => {}".format(bestAccuracyMLP*100))
    print("Testing Accuracy => {}\n".format(accuracy_score(yTest, yPredMLP)*100))
    print("Confusion Matrix => \n{}\n".format(confusion_matrix(yTest, yPredMLP)))
    print("Classification Summary => \n{}\n".format(classification_report(yTest, yPredMLP)))
    plt.scatter(yTest, yPredMLP)

mlp(df, scaledData, targetVariable)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve

# Below function is taken from the official documentation of "sklearn" 
# Link : http://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=10,
                        n_jobs=10, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


title = "Learning Curves"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)
xTrain, xTest, yTrain, yTest = train_test_split(scaledData, targetVariable, test_size=0.2, random_state=0, shuffle=False)
xTrain = xTrain.sort_index()
xTest = xTest.sort_index()
yTrain = yTrain.sort_index()
yTest = yTest.sort_index()

estimator = MLPClassifier(hidden_layer_sizes=(100, 200), alpha=0.1, max_iter=500)
plot_learning_curve(estimator, title, xTrain, yTrain, cv=10, n_jobs=10)

From all the models that I have tried like Decision Trees, KNN, Logistic Regression and Multilayer Perceptron, we can clearly say that Multilayer Perceptron acts as the best model for the dataset provided. 

### Some Interesting Insights drawn from the Dataset and Models<br>
**Why 'Gender' is related to 'Life Struggles'?**
To answer the question of why 'Life Struggles' is related to 'Gender', I would first point to the strong correlation in 'Gender' with 'height' and 'weight'. Thus, we can say that 'Life Struggles' is strongly correlated with 'height' and 'weight'. Women are shorter in height and lighter in weight than men. The original phrase for 'Life Struggles' is 'I cry when I feel down or things don't go the right way'. Since, womenn used this phrase a lot in the dataset, 'Life Struggles' and 'Gender' are said to be correlated strongly.<br><br>
**Why 'Gender' is related to 'Western'?**
Unlike the previous argument, in case of correlation between 'Western' and 'Gender', from in-depth analysis of data, we get to know that 75% of the people have voted more than 3 when asked whether they like 'Western Movies' and majority of the people have voted 5 for the question. These statistics show that most of the people like 'Western Movies' irrespective of the 'Gender'.<br><br>
**Is the Multilayer Perceptron really learning or just memorizing the dataset?**
This question is really an interesting one. As you increase the hidden layers and the neurons in those hidden layers, you clearly find that the training and testing accuracy shoot up. I tried to understand why this phenonmenon is happening and got to know of the memorization capabilities of the deep neural networks from a Technical paper published by several Deep Networks scholars from MIT and Google. The link to the paper is <a href="https://arxiv.org/pdf/1706.05394.pdf">this</a>. By understanding how the deep networks are capable of memorizing the datasets with relatively smaller size like the one used in this project. In order to get the best accuracy, I would say Deep Neural Networks are the best for such datasets. But I also tend to doubt my model to a certain extent because it is a sheer memorization process and not an actual training that is happening. Because the scope of the project was not very huge, I would not dig much deeper into it and conclude that using Deep Neural Networks like a Multilayer Perceptron is the best strategy to gain maximum advantage. 